# Spotify Suggestion Model V2

## Infrastructure

In [ ]:
import sys
# !{sys.executable} -m pip install spotipy

In [ ]:
username = "agwx2"
cid = "17c11cf9271f4980a85e6aab83cba6bb" 
secret = "015c9abf126340a7981e7ba64dbe7828"
good_playlist_id = "61BmnG6d7ifmj8Dr5tuCBK"
bad_playlist_id = "53FPslFSIAGvVyERcNG4J3"
source_playlist_id = "5ghFHvZmV4FDK5YLeZQRu9"

In [ ]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split


from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LinearRegression

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

## Data Collection

In [ ]:
sp_cred = SpotifyClientCredentials(
    client_id=cid,
    client_secret=secret
)
sp = spotipy.Spotify(client_credentials_manager=sp_cred)

In [ ]:
good_playlist = sp.user_playlist(username, good_playlist_id)
good_tracks = good_playlist["tracks"]
good_songs = good_tracks["items"]

while good_tracks['next']:
    good_tracks = sp.next(good_tracks)
    for item in good_tracks["items"]:
        good_songs.append(item)

good_ids = []
for i in range(len(good_songs)):
    good_ids.append(good_songs[i]['track']['id'])
print(len(good_ids))
print(good_ids)

132
['4DMKwE2E2iYDKY01C335Uw', '1QEEqeFIZktqIpPI4jSVSF', '5QTxFnGygVM4jFQiBovmRo', '05GhiSRMv34B1N6BsixbmL', '2V0yBHMWB8GoMm80AXH4ot', '7GqWnsKhMtEW0nzki5o0d8', '1EID0aupOLdv4ZDQu2Jbrz', '790YJcgHlN3SaosQCHlWzn', '7tFiyTwD0nx5a1eklYtX2J', '6cFZ4PLC19taNlpl9pbGMf', '54flyrjcdnQdco7300avMJ', '7ccI9cStQbQdystvc6TvxD', '4igIYHF3B5VBxEafHauVo3', '7hQJA50XrCWABAu5v6QZ4i', '57JVGBtBLCfHw2muk5416J', '35ItUJlMtjOQW3SSiTCrrw', '11IzgLRXV7Cgek3tEgGgjw', '2htmMitogin6NMGABye6YG', '59CsbfwLNCAUGbBQV3Tki4', '78lgmZwycJ3nzsdgmPPGNx', '4PRGxHpCpF2yoOHYKQIEwD', '5CQ30WqJwcep0pYcV4AMNc', '3BGwxsmbbOe2jmZfh5CUAO', '24NwBd5vZ2CK8VOQVnqdxr', '5SZ6zX4rOrEQferfFC2MfP', '4iihDHIoKZdOeOW1kFDUtR', '0c1gHntWjKD7QShC8s99sq', '5MxNLUsfh7uzROypsoO5qe', '3ZqnF9T5vtvIPqHmloNb0o', '0ZGLuduCPjgWY1n85ykgMe', '64yPXtpdrGRsFKc06nb39l', '5svBTKTKa8S8jaZqV3XZgv', '0sjPYRacvarCBduZaKdbJW', '7K2e29mEjxVx4Dwbiqtrji', '0nHfNZYzI3cXMgc4pN4kRi', '5voO100jVn9PK4NUETgM8W', '4u9f8hqstB7iITDJNzKhQx', '5wzYIbYrLZ3OiLCUyYISc2', '5zOoNB

In [ ]:
bad_playlist = sp.user_playlist(username, bad_playlist_id)
bad_tracks = bad_playlist["tracks"]
bad_songs = bad_tracks["items"]

while bad_tracks['next']:
    bad_tracks = sp.next(bad_tracks)
    for item in bad_tracks["items"]:
        bad_songs.append(item)

bad_ids = []
for i in range(len(bad_songs)):
    bad_ids.append(bad_songs[i]['track']['id'])
print(len(bad_ids))
print(bad_ids)

217
['5KVciTEfXvpcFHR3RFbtJy', '6iyp6udVpLuq3MCUulZT4h', '6YyvzSFCx1qogS5VWfXc6B', '7iaw359G2XT14uTfV9feip', '51rXHuKN8Loc4sUlKPODgH', '42XnD59SFcomCJu55EPhqJ', '5tFep7dXGd7vEJ668wTPux', '40oKW22ZNNkEdZLJTScaQI', '7DuvVy7JfFgXwWlVzyyPys', '0ESJlaM8CE1jRWaNtwSNj8', '4bJX21zvsGJELwVNf902qk', '07pJdL0L9XDGW0HBj0rIS2', '1cTZMwcBJT0Ka3UJPXOeeN', '6urLjX35oXXRYtcH1b3dCh', '6HmWy7ttkeZGG2KEtt8AqX', '1yxgsra98r3qAtxqiGZPiX', '3be3kLKYYLtAb8WERUM1sS', '4DTpngLjoHj5gFxEZFeD3J', '4qKcDkK6siZ7Jp1Jb4m0aL', '24lORMRGMv9sXpZJdN1PVm', '6Vur2D5ks3pdAZqqmLIeab', '2eMwDehkIC1j68U6FA3Eiq', '2yUbCEiaolfSMluDo9RMmG', '2nYwV62UABTH0NanrwVIdQ', '263le58t0LrdRlCZDcGQXd', '795qeY9x7NmfYJOZ9dQY0Y', '1xfFPJdRjgDAaMcDgbXwyh', '08Tbfp1GXta9Gb4RhoTuoX', '0H8XeaJunhvpBdBFIYi6Sh', '4fndbjoz1qJyK6JcLdKfzm', '5jyyPsIGM2yqkZN9R3TmvN', '4vIhvFLrnDD9fEjpjWs0Ky', '2D2w9943rsnJOGCrI4aMQp', '628Ueb4sRiXApObhcU9iPU', '1pF3VcWYHeuVPflRwGBaWV', '2Bv0MbaqYx2IrQa7Z6xBT0', '2Jlx7CGCVi3mRbKcA53YOn', '1cZlBZwnwGPtYeRIeQcoFh', '4TirSu

In [ ]:
source_playlist = sp.user_playlist(username, source_playlist_id)
source_tracks = source_playlist["tracks"]
source_songs = source_tracks["items"]

while source_tracks['next']:
    source_tracks = sp.next(source_tracks)
    for item in source_tracks["items"]:
        source_songs.append(item)

source_ids = []
for i in range(len(source_songs)):
    if source_songs[i]['track'] != None:
      source_ids.append(source_songs[i]['track']['id'])
print(len(source_ids))
print(source_ids)

122
['0Dm43YLUlGdePpSbI1ct8h', '70eFcWOvlMObDhURTqT4Fv', '1DhRbox3xkceP64k3JeYfW', '3FHXUqaUer9o231Je1zxIN', '2md2i5QvelRFnafpnd6LOg', '1z3L82jeYnTmYUjPc7zXcu', '1MJ5f5EYBC92ADD6xcz7nb', '0W0R48uImgPVVJgb5YTC3I', '0nMqu5vN44GNv8UOQxmBJK', '5pgZpHqfv4TSomtkfGZGrG', '14rWEXQGBsFB7ALUc8G4sr', '4sqIBV3ukQDqRr8Me2nvQb', '1QvpsbjYAVVFvFrWVC0qnQ', '6Wz9rIfo9tOBcVCd1Mo7F7', '3OUyyDN7EZrL7i0Sbi5SVd', '32q9OuCp7wFdLZKuCXm0wl', '5JWPUEov2wlX7c0jhYZpeB', '2BITQ360Knh6qNAOqR7Dyq', '5O2Yjlcs5pSpHq1gcA0cwg', '23bOfmnZiN42pB1frXNyak', '1LwwZ0xoUz5gR8XrxEUKAK', '17aVK3oQdgwVG2l0HpP67W', '25nzKGDiua1lE9Qo5V19GL', '60dwJ8dnqZ4WrIdiu7LkNR', '7KrlNS2VqiNNJ82RMQdxBn', '2xdwCJLfKCkat14mlgR26M', '3Jbpo2slhW2bbVneiJnC9G', '4dVbhS6OiYvFikshyaQaCN', '5UWwZ5lm5PKu6eKsHAGxOk', '67vYdAAM3oGsEImCRfbtsF', '3ptB4SmuSJTGhQByFrMJVJ', '2CDVqsGGkf2v1tMETt3fq5', '4QMSoa7Fh3y4kmMNuBtOgt', '1F9kEuwT29fgqlovIclu81', '0EC0kuRKIUlru5eRxWYYyv', '6KYVWcOQWm7JrlsPMkrkNi', '732Sqj2piYPhdMPENP4axz', '0ooz46SAYMJsdOPnwW4nKK', '1mSClO

In [ ]:
training_features = []

for i in range(0, len(good_ids), 50):
    audio_features = sp.audio_features(good_ids[i : i + 50])
    for track in audio_features:
        training_features.append(track)
        training_features[-1]['target'] = 1

for i in range(0, len(bad_ids), 50):
    audio_features = sp.audio_features(bad_ids[i : i + 50])
    for track in audio_features:
        training_features.append(track)
        training_features[-1]['target'] = 0

print(len(training_features))
print(training_features)

349
[{'danceability': 0.482, 'energy': 0.786, 'key': 7, 'loudness': -5.735, 'mode': 1, 'speechiness': 0.032, 'acousticness': 0.00321, 'instrumentalness': 8.88e-05, 'liveness': 0.446, 'valence': 0.793, 'tempo': 126.755, 'type': 'audio_features', 'id': '4DMKwE2E2iYDKY01C335Uw', 'uri': 'spotify:track:4DMKwE2E2iYDKY01C335Uw', 'track_href': 'https://api.spotify.com/v1/tracks/4DMKwE2E2iYDKY01C335Uw', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4DMKwE2E2iYDKY01C335Uw', 'duration_ms': 323000, 'time_signature': 4, 'target': 1}, {'danceability': 0.377, 'energy': 0.681, 'key': 7, 'loudness': -8.039, 'mode': 1, 'speechiness': 0.0298, 'acousticness': 0.00088, 'instrumentalness': 0.0023, 'liveness': 0.0504, 'valence': 0.285, 'tempo': 108.789, 'type': 'audio_features', 'id': '1QEEqeFIZktqIpPI4jSVSF', 'uri': 'spotify:track:1QEEqeFIZktqIpPI4jSVSF', 'track_href': 'https://api.spotify.com/v1/tracks/1QEEqeFIZktqIpPI4jSVSF', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1QEEqeFI

In [ ]:
source_features = []
for i in range(0, len(source_ids), 50):
    audio_features = sp.audio_features(source_ids[i : i + 50])
    for track in audio_features:
        source_features.append(track)
        source_features[-1]['target'] = 0  # arbitrary

print(len(source_features))
print(source_features)

122
[{'danceability': 0.662, 'energy': 0.624, 'key': 10, 'loudness': -13.606, 'mode': 0, 'speechiness': 0.0417, 'acousticness': 0.165, 'instrumentalness': 0, 'liveness': 0.0674, 'valence': 0.961, 'tempo': 174.513, 'type': 'audio_features', 'id': '0Dm43YLUlGdePpSbI1ct8h', 'uri': 'spotify:track:0Dm43YLUlGdePpSbI1ct8h', 'track_href': 'https://api.spotify.com/v1/tracks/0Dm43YLUlGdePpSbI1ct8h', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0Dm43YLUlGdePpSbI1ct8h', 'duration_ms': 252560, 'time_signature': 4, 'target': 0}, {'danceability': 0.64, 'energy': 0.647, 'key': 5, 'loudness': -8.114, 'mode': 0, 'speechiness': 0.186, 'acousticness': 0.124, 'instrumentalness': 0, 'liveness': 0.0801, 'valence': 0.546, 'tempo': 92.979, 'type': 'audio_features', 'id': '70eFcWOvlMObDhURTqT4Fv', 'uri': 'spotify:track:70eFcWOvlMObDhURTqT4Fv', 'track_href': 'https://api.spotify.com/v1/tracks/70eFcWOvlMObDhURTqT4Fv', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/70eFcWOvlMObDhURTqT4Fv'

## Data Preparation

In [ ]:
trainingData = pd.DataFrame(training_features)

In [ ]:
trainingData

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,target
0,0.482,0.786,7,-5.735,1,0.0320,0.00321,0.000089,0.4460,0.7930,126.755,audio_features,4DMKwE2E2iYDKY01C335Uw,spotify:track:4DMKwE2E2iYDKY01C335Uw,https://api.spotify.com/v1/tracks/4DMKwE2E2iYD...,https://api.spotify.com/v1/audio-analysis/4DMK...,323000,4,1
1,0.377,0.681,7,-8.039,1,0.0298,0.00088,0.002300,0.0504,0.2850,108.789,audio_features,1QEEqeFIZktqIpPI4jSVSF,spotify:track:1QEEqeFIZktqIpPI4jSVSF,https://api.spotify.com/v1/tracks/1QEEqeFIZktq...,https://api.spotify.com/v1/audio-analysis/1QEE...,285133,4,1
2,0.333,0.927,9,-8.550,0,0.0733,0.00290,0.000208,0.2970,0.3850,141.466,audio_features,5QTxFnGygVM4jFQiBovmRo,spotify:track:5QTxFnGygVM4jFQiBovmRo,https://api.spotify.com/v1/tracks/5QTxFnGygVM4...,https://api.spotify.com/v1/audio-analysis/5QTx...,308120,4,1
3,0.316,0.320,9,-13.045,1,0.0387,0.40800,0.000067,0.0925,0.2940,74.163,audio_features,05GhiSRMv34B1N6BsixbmL,spotify:track:05GhiSRMv34B1N6BsixbmL,https://api.spotify.com/v1/tracks/05GhiSRMv34B...,https://api.spotify.com/v1/audio-analysis/05Gh...,250933,3,1
4,0.341,0.293,9,-12.408,1,0.0356,0.75600,0.000023,0.0833,0.1780,93.294,audio_features,2V0yBHMWB8GoMm80AXH4ot,spotify:track:2V0yBHMWB8GoMm80AXH4ot,https://api.spotify.com/v1/tracks/2V0yBHMWB8Go...,https://api.spotify.com/v1/audio-analysis/2V0y...,249213,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,0.567,0.463,6,-11.669,0,0.5040,0.73200,0.000000,0.7090,0.3240,126.982,audio_features,40IQooNkPRVtn4zlE3ZFpA,spotify:track:40IQooNkPRVtn4zlE3ZFpA,https://api.spotify.com/v1/tracks/40IQooNkPRVt...,https://api.spotify.com/v1/audio-analysis/40IQ...,496173,4,0
345,0.554,0.498,9,-8.866,1,0.0685,0.23000,0.000006,0.7950,0.4130,79.635,audio_features,5hVghJ4KaYES3BFUATCYn0,spotify:track:5hVghJ4KaYES3BFUATCYn0,https://api.spotify.com/v1/tracks/5hVghJ4KaYES...,https://api.spotify.com/v1/audio-analysis/5hVg...,190067,4,0
346,0.598,0.252,2,-15.086,1,0.0644,0.19400,0.133000,0.0992,0.0352,120.064,audio_features,4S7YHmlWwfwArgd8LfSPud,spotify:track:4S7YHmlWwfwArgd8LfSPud,https://api.spotify.com/v1/tracks/4S7YHmlWwfwA...,https://api.spotify.com/v1/audio-analysis/4S7Y...,238307,4,0
347,0.621,0.601,6,-5.616,0,0.1480,0.05220,0.000000,0.4600,0.4570,116.735,audio_features,696DnlkuDOXcMAnKlTgXXK,spotify:track:696DnlkuDOXcMAnKlTgXXK,https://api.spotify.com/v1/tracks/696DnlkuDOXc...,https://api.spotify.com/v1/audio-analysis/696D...,163636,5,0


In [ ]:
train, test = train_test_split(trainingData, test_size = 0.15)

In [ ]:
target_features = ["danceability", "loudness", "valence", "energy", "instrumentalness", "acousticness", "key", "speechiness", "duration_ms"]

x_train = train[target_features]
y_train = train["target"]
x_test = test[target_features]
y_test = test["target"]

In [ ]:
x_train

,danceability,loudness,valence,energy,instrumentalness,acousticness,key,speechiness,duration_ms
19,0.564,-10.068,0.619,0.932,0.16900,0.013000,11,0.0554,146250
96,0.554,-7.126,0.598,0.790,0.00401,0.000738,9,0.0695,202627
99,0.313,-6.215,0.300,0.882,0.33800,0.034600,0,0.0355,431093
165,0.668,-4.151,0.330,0.844,0.00879,0.003890,1,0.0956,172017
200,0.582,-6.131,0.155,0.681,0.00000,0.009820,7,0.0467,219840
...,...,...,...,...,...,...,...,...,...
166,0.842,-5.867,0.184,0.609,0.00000,0.044300,11,0.1900,267627
295,0.813,-6.941,0.191,0.453,0.00000,0.093300,1,0.3380,208027
232,0.570,-7.562,0.270,0.703,0.00000,0.020000,6,0.2220,180016
73,0.574,-4.793,0.423,0.913,0.00158,0.061000,6,0.1330,208400


In [ ]:
y_train

19     1
96     1
99     1
165    0
200    0
      ..
166    0
295    0
232    0
73     1
328    0
Name: target, Length: 296, dtype: int64

In [ ]:
x_test

,danceability,loudness,valence,energy,instrumentalness,acousticness,key,speechiness,duration_ms
54,0.671,-8.144,0.9710,0.669,0.892000,0.046000,2,0.0687,245600
195,0.846,-8.059,0.4410,0.538,0.000000,0.027200,1,0.0576,208160
270,0.832,-5.721,0.0561,0.491,0.000013,0.015700,8,0.1090,135493
35,0.394,-10.996,0.1820,0.644,0.198000,0.244000,5,0.0367,301560
95,0.610,-6.046,0.8190,0.824,0.000002,0.013600,4,0.0534,210133
66,0.537,-3.002,0.8280,0.959,0.000002,0.070400,0,0.0557,222707
203,0.538,-8.794,0.4090,0.624,0.000002,0.000175,11,0.3250,127427
160,0.620,-6.508,0.3310,0.690,0.000000,0.093100,7,0.0493,234213
55,0.336,-10.875,0.4270,0.792,0.001750,0.003510,9,0.0840,355304
346,0.598,-15.086,0.0352,0.252,0.133000,0.194000,2,0.0644,238307


In [ ]:
y_test

54     1
195    0
270    0
35     1
95     1
66     1
203    0
160    0
55     1
346    0
167    0
136    0
21     1
179    0
310    0
93     1
164    0
173    0
61     1
274    0
16     1
125    1
56     1
79     1
225    0
334    0
62     1
294    0
100    1
342    0
228    0
258    0
159    0
192    0
26     1
77     1
279    0
335    0
267    0
245    0
0      1
11     1
126    1
243    0
339    0
299    0
170    0
171    0
204    0
133    0
287    0
291    0
84     1
Name: target, dtype: int64

## Model Evaluation

### Gradient Boosting

In [ ]:
print(x_train.columns)

Index(['danceability', 'loudness', 'valence', 'energy', 'instrumentalness',
       'acousticness', 'key', 'speechiness', 'duration_ms'],
      dtype='object')


In [ ]:
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=.1, max_depth=1, random_state=2)
gbc.fit(x_train, y_train)
gbc_pred = gbc.predict(x_test)

score = accuracy_score(y_test, gbc_pred) * 100
print("Accuracy using Gradient Boost: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, gbc_pred))

Accuracy using Gradient Boost:  92.45 %
Mean squared error: 0.08


### Neural Networks

#### Baseline

In [ ]:
def keras_nn_baseline_model():
  model = Sequential()
  model.add(Dense(9, input_dim=9, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [ ]:
keras_nn = KerasClassifier(build_fn=keras_nn_baseline_model, epochs=100, batch_size=5, verbose=0)

In [ ]:
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(keras_nn, x_train, y_train, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 52.67% (11.93%)


In [ ]:
keras_nn.fit(x_train, y_train)

#### Improved

In [ ]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=keras_nn_baseline_model, epochs=100, batch_size=5, verbose=0)))
s = Pipeline(estimators)

In [ ]:
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, x_train, y_train, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean() * 100, results.std() * 100))

Standardized: 96.31% (2.77%)


## Generate Predictions

In [ ]:
sourceData = pd.DataFrame(source_features)
sourceData = sourceData[target_features]

In [ ]:
sourceData

,danceability,loudness,valence,energy,instrumentalness,acousticness,key,speechiness,duration_ms
0,0.662,-13.606,0.9610,0.624,0.000000,0.16500,10,0.0417,252560
1,0.640,-8.114,0.5460,0.647,0.000000,0.12400,5,0.1860,197867
2,0.716,-5.313,0.9100,0.823,0.000000,0.13200,5,0.0495,161747
3,0.468,-6.536,0.7620,0.630,0.000000,0.31000,8,0.0843,204507
4,0.397,-4.577,0.5560,0.903,0.000002,0.00575,2,0.0457,201373
...,...,...,...,...,...,...,...,...,...
117,0.519,-9.982,0.4880,0.181,0.000000,0.94400,5,0.0795,82147
118,0.615,-8.615,0.7470,0.592,0.955000,0.04880,0,0.0443,263673
119,0.512,-5.128,0.8380,0.955,0.000013,0.17200,7,0.3540,40227
120,0.807,-9.729,0.9610,0.510,0.115000,0.04100,0,0.0433,279773


In [ ]:
model = gbc
# model = keras_nn

In [ ]:
pred = model.predict(sourceData)

In [ ]:
source_playlist_songs[0]

{'added_at': '2020-10-01T05:24:42Z',
 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/agwx2'},
  'href': 'https://api.spotify.com/v1/users/agwx2',
  'id': 'agwx2',
  'type': 'user',
  'uri': 'spotify:user:agwx2'},
 'is_local': False,
 'primary_color': None,
 'track': None,
 'video_thumbnail': None}

In [ ]:
source_playlist_songs = source_playlist['tracks']['items']

print("ACCEPTED")

i = 0
for prediction in pred:
    if prediction == 1:
        song_title = source_playlist_songs[i + 1]['track']['name']
        artist_name = source_playlist_songs[i + 1]['track']['artists'][0]['name']
        print ("Song: " + song_title + ", By: "+ artist_name)
    i = i + 1


print("REJECTED")

i = 0
for prediction in pred:
    if prediction == 0:
        song_title = source_playlist_songs[i + 1]['track']['name']
        artist_name = source_playlist_songs[i + 1]['track']['artists'][0]['name']
        print ("Song: " + song_title + ", By: "+ artist_name)
    i = i + 1

ACCEPTED
Song: Meant to Live, By: Switchfoot
Song: Meant To Live - Acoustic Version, By: Switchfoot
Song: Chamber Of Reflection, By: Mac DeMarco
Song: Wavy, By: Janko Nilovic
Song: Malibu 1992, By: COIN
Song: Soap On Your Skin, By: The Solids
Song: Across The Overpass, By: The Solids
Song: Hey Beautiful - From "How I Met Your Mother"/Theme, By: The Solids
Song: Twenty Something, By: Nightly
Song: BYE, By: Jaden
Song: Lost In Yesterday, By: Tame Impala
Song: Starchild, By: Burn Pilot
Song: Young Blood, By: The Naked And Famous
Song: Saturday, By: Twenty One Pilots
Song: My Hero, By: Foo Fighters
Song: Everlong, By: Foo Fighters
Song: Times Like These, By: Foo Fighters
Song: the angel of 8th ave., By: Gang of Youths
Song: love race (feat. Kellin Quinn), By: Machine Gun Kelly
Song: Enghave, By: Marco Egeberg
Song: Relax, By: Frankie Goes To Hollywood
Song: Rockstar, By: Mallrat
Song: Galaxies, By: Laura Veirs
Song: Over Now (with The Weeknd), By: Calvin Harris
Song: If I Got It (Your Love